# Weight & Biases (WandB) module

In [ ]:
import wandb

api = wandb.Api()

#### Get configuration details of any run

In [ ]:
entity, project, run_id = "ammar20112001-ml", "Attention-Is-All-You-Need--reproduced", "uyow0rcm"
run = api.run(f"{entity}/{project}/{run_id}")

print('CONFIG')
for key, value in run.config.items():
  print(key, ':', value)
print('\nSUMMARY')
for key, value in run.summary.items():
  print(key, ':', value)
print('\nGit commit:', run.commit, '\n')
run.history()

#### Read metrics from a run

In [ ]:
run = api.run(f"{entity}/{project}/{run_id}")
if run.state == "finished":
    for i, row in run.history().iterrows():
        print(row["_timestamp"], row["LOSS_step"])

#### Compare two runs

In [ ]:
# replace with your <entity>, <project>, and <run_id>
run1 = api.run(f"{entity}/{project}/uyow0rcm")
run2 = api.run(f"{entity}/{project}/1mj38s4b")


df = pd.DataFrame([run1.config, run2.config]).transpose()

df.columns = [run1.name, run2.name]
print(df[df[run1.name] != df[run2.name]])

#### Export experiment manually

In [ ]:
import random

n_epochs = 5
config = {"n_epochs": n_epochs}
run = wandb.init(project=project, config=config)
for n in range(run.config.get("n_epochs")):
    run.log(
        {"val": random.randint(0, 1000), "loss": (random.randint(0, 1000) / 1000.00)}
    )
run.finish()

#### Querying Multiple Runs

In [ ]:
import pandas as pd

entity, project = "ammar20112001-ml", "Attention-Is-All-You-Need--reproduced"

runs = api.runs(entity + "/" + project)

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)

runs_df.to_csv("project.csv")

#### Update metrics for a run, after the run has finished

In [ ]:
run = api.run(f"{entity}/{project}/1mj38s4b")
run.summary["loss"] = 100
#run.summary["accuracy_histogram"] = wandb.Histogram(numpy_array)
run.summary.update()

#### Getting artifacts

In [ ]:
entity, project, run_id = "ammar20112001-ml", "Attention-Is-All-You-Need--reproduced", "a403ono3"
run = api.run(f"{entity}/{project}/{run_id}")

hist = run.history()  # and pull down a sample of the data as a pandas DataFrame

# which artifacts where created and logged?
artifacts = run.logged_artifacts()

for artifact in artifacts:
    print(f"artifact of type {artifact.type}: {artifact.name}")